# コーナー検出（Harris、Shi-Tomasi）

このノートブックでは、コーナー検出（harris、shi-tomasi）について学びます。

## 目次
1. [Harrisコーナー検出](#harrisコーナー検出)
2. [Shi-Tomasiコーナー検出](#shi-tomasiコーナー検出)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from skimage import filters, feature

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["axes.unicode_minus"] = False
np.random.seed(42)
print("ライブラリのインポートが完了しました。")

## Harrisコーナー検出

In [ ]:
# Harrisコーナー検出
# 画像内のコーナー（角）を検出します

# サンプル画像の作成（コーナーを含む）
img = np.zeros((200, 200), dtype=np.uint8)
img[50:150, 50:150] = 255  # 白い四角
img = cv2.GaussianBlur(img, (5, 5), 0)

# グレースケールに変換（既にグレースケールだが、形式を統一）
gray = img.astype(np.float32)

# Harrisコーナー検出
harris_response = cv2.cornerHarris(gray, blockSize=2, ksize=3, k=0.04)

# 閾値処理
harris_thresh = 0.01 * harris_response.max()
harris_corners = np.zeros_like(img)
harris_corners[harris_response > harris_thresh] = 255

# コーナー位置の取得
corner_coords = np.argwhere(harris_response > harris_thresh)

# 結果の表示
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(img, cmap='gray')
axes[0].set_title('元の画像')
axes[0].axis('off')

axes[1].imshow(harris_response, cmap='hot')
axes[1].set_title('Harris応答')
axes[1].axis('off')

# コーナーをマーク
img_with_corners = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
for y, x in corner_coords:
    cv2.circle(img_with_corners, (x, y), 3, (0, 255, 0), -1)

axes[2].imshow(img_with_corners)
axes[2].set_title(f'検出されたコーナー ({len(corner_coords)}個)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print(f"検出されたコーナー数: {len(corner_coords)}")
print("Harrisコーナー検出のパラメータ:")
print("- blockSize: 近傍領域のサイズ")
print("- ksize: Sobel演算子のカーネルサイズ")
print("- k: Harris検出器の自由パラメータ（通常0.04-0.06）")

## Shi-Tomasiコーナー検出

In [ ]:
# Shi-Tomasiコーナー検出（Good Features to Track）
# Harrisの改良版で、より安定したコーナー検出

# Shi-Tomasiコーナー検出
corners = cv2.goodFeaturesToTrack(gray, maxCorners=100, qualityLevel=0.01, minDistance=10)

# コーナーを描画
img_shitomasi = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
if corners is not None:
    corners = np.int0(corners)
    for corner in corners:
        x, y = corner.ravel()
        cv2.circle(img_shitomasi, (x, y), 3, (0, 255, 0), -1)

# 異なるパラメータで比較
corners_high_quality = cv2.goodFeaturesToTrack(gray, maxCorners=100, qualityLevel=0.1, minDistance=10)
corners_many = cv2.goodFeaturesToTrack(gray, maxCorners=200, qualityLevel=0.01, minDistance=5)

img_high_quality = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
if corners_high_quality is not None:
    corners_high_quality = np.int0(corners_high_quality)
    for corner in corners_high_quality:
        x, y = corner.ravel()
        cv2.circle(img_high_quality, (x, y), 3, (0, 255, 0), -1)

img_many = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
if corners_many is not None:
    corners_many = np.int0(corners_many)
    for corner in corners_many:
        x, y = corner.ravel()
        cv2.circle(img_many, (x, y), 3, (0, 255, 0), -1)

# 結果の表示
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('元の画像')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_shitomasi)
axes[0, 1].set_title(f'Shi-Tomasi (quality=0.01, {len(corners) if corners is not None else 0}個)')
axes[0, 1].axis('off')

axes[1, 0].imshow(img_high_quality)
axes[1, 0].set_title(f'Shi-Tomasi (quality=0.1, {len(corners_high_quality) if corners_high_quality is not None else 0}個)')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_many)
axes[1, 1].set_title(f'Shi-Tomasi (maxCorners=200, {len(corners_many) if corners_many is not None else 0}個)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("Shi-Tomasiコーナー検出のパラメータ:")
print("- maxCorners: 検出する最大コーナー数")
print("- qualityLevel: コーナーの品質レベル（0-1、高いほど高品質）")
print("- minDistance: コーナー間の最小距離（ピクセル）")
print("Harrisよりも安定した結果を提供します。")